## RAG Day 3

### Expert Question Answerer for InsureLLM

LangChain 1.0 implementation of a RAG pipeline.

Using the VectorStore we created last time (with HuggingFace `all-MiniLM-L6-v2`)

In [11]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [10]:
MODEL = "gpt-5-mini"
DB_NAME = "vector_db"
load_dotenv(override=True)

True

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME,embedding_function=embeddings)


In [13]:
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0,model_name=MODEL)

In [14]:
retriever.invoke("who is Avery?")

[Document(id='60f1cb17-c71e-415b-b7d0-8eade8bf75db', metadata={'doc_type': 'employees', 'source': 'knowledge-base\\employees\\Avery Lancaster.md'}, page_content="## Other HR Notes\n- **Professional Development**: Avery has actively participated in leadership training programs and industry conferences, representing Insurellm and fostering partnerships.  \n- **Diversity & Inclusion Initiatives**: Avery has championed a commitment to diversity in hiring practices, seeing visible improvements in team representation since 2021.  \n- **Work-Life Balance**: Feedback revealed concerns regarding work-life balance, which Avery has approached by implementing flexible working conditions and ensuring regular check-ins with the team.\n- **Community Engagement**: Avery led community outreach efforts, focusing on financial literacy programs, particularly aimed at underserved populations, improving Insurellm's corporate social responsibility image.  \n\nAvery Lancaster has demonstrated resilience and a

In [15]:
llm.invoke("Who is Avery Lancaster?")

AIMessage(content='I don’t have enough context — there are multiple people who could have that name. Do you mean a specific Avery Lancaster (for example a public figure, fictional character, author, athlete, or someone you saw on social media)? \n\nIf you tell me where you saw the name or paste a link, I can summarize who they are. If you want to look them up yourself, try searching:\n- "Avery Lancaster" plus the context (e.g., book title, company, location)\n- LinkedIn for professional profiles\n- Twitter/Instagram/Facebook for social profiles\n- News search or Google News for press coverage\n\nWhich one do you mean?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 590, 'prompt_tokens': 11, 'total_tokens': 601, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'opena

In [16]:
SYSTEM_PROMPT_TEMPLATE = """
You are knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [21]:
def answer_question(question: str,history):
     docs = retriever.invoke(question)
     print(f"[DEBUG] retrieved docs are {docs}")
     context = "\n\n".join(doc.page_content for doc in docs)
     system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
     response =llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
     return response.content


In [22]:
answer_question("Who is Averi Lancaster?", [])

[DEBUG] retrieved docs are [Document(id='05ab354b-9594-40e2-bbd0-b34efb57ce7b', metadata={'source': 'knowledge-base\\employees\\Emily Carter.md', 'doc_type': 'employees'}, page_content="Emily Carter exemplifies the kind of talent that drives Insurellm's success and is an invaluable asset to the company."), Document(id='224ddaa6-ccd2-4383-843d-5a4e7b2015c2', metadata={'source': 'knowledge-base\\employees\\Rachel Martinez.md', 'doc_type': 'employees'}, page_content='## Other HR Notes\n- **Professional Development:** Completed Product Leadership Certification from Pragmatic Institute (2021). Regular attendee at ProductCon and InsurTech conferences.\n- **Mentorship:** Currently mentors two Associate Product Managers and actively participates in the Women in Tech initiative at Insurellm.\n- **Skills:** Expert in Agile/Scrum methodologies, JIRA, Figma, SQL, and product analytics tools.\n- **Feedback:** Known for her strategic thinking and ability to balance customer needs with business objec

'Do you mean Avery Lancaster (spelled A-v-e-r-y)? If so:\n\n- Avery Lancaster is the co-founder and Chief Executive Officer (CEO) of Insurellm.  \n- Born March 15, 1985; based in San Francisco, California.  \n- Co-founded Insurellm in 2015 and has led the company since then, credited with innovative leadership and risk-management expertise that helped grow the company.  \n- Prior to founding Insurellm, Avery was Senior Product Manager at Innovate Insurance Solutions (2013–2015).  \n- Current listed salary: $225,000.\n\nIf you meant a different person or want more detail about Avery’s background or role, tell me what you’d like to know.'

In [23]:
gr.ChatInterface(answer_question).launch()

c:\Users\rambo\projects\ED\llm_engineering\.venv\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[DEBUG] retrieved docs are [Document(id='05ab354b-9594-40e2-bbd0-b34efb57ce7b', metadata={'source': 'knowledge-base\\employees\\Emily Carter.md', 'doc_type': 'employees'}, page_content="Emily Carter exemplifies the kind of talent that drives Insurellm's success and is an invaluable asset to the company."), Document(id='224ddaa6-ccd2-4383-843d-5a4e7b2015c2', metadata={'doc_type': 'employees', 'source': 'knowledge-base\\employees\\Rachel Martinez.md'}, page_content='## Other HR Notes\n- **Professional Development:** Completed Product Leadership Certification from Pragmatic Institute (2021). Regular attendee at ProductCon and InsurTech conferences.\n- **Mentorship:** Currently mentors two Associate Product Managers and actively participates in the Women in Tech initiative at Insurellm.\n- **Skills:** Expert in Agile/Scrum methodologies, JIRA, Figma, SQL, and product analytics tools.\n- **Feedback:** Known for her strategic thinking and ability to balance customer needs with business objec